In [9]:
# @title RUN LIBRARIES
import ee
import geemap
import calendar
from datetime import datetime

In [10]:
# @title AUTHENTICATE GOOGLE EARTH ENGINE
ee.Authenticate()
ee.Initialize(project='ee-sharon')

In [11]:
# @title FUNCTIONS
class Sentinel2Exporter:
    def __init__(self, coords_list, start_date, end_date, max_cloud=12):
        self.coords_list = coords_list
        self.start_date = start_date
        self.end_date = end_date
        self.max_cloud = max_cloud
        self.month_names = [
            'Enero','Febrero','Marzo','Abril','Mayo','Junio',
            'Julio','Agosto','Septiembre','Octubre','Noviembre','Diciembre'
        ]
        self.roi_fc = self._crear_feature_collection()
        self.months = self._generar_rangos_mensuales()

    def _crear_feature_collection(self):
        features = [
            ee.Feature(ee.Geometry.Polygon(coords), {'id': i + 1})
            for i, coords in enumerate(self.coords_list)
        ]
        return ee.FeatureCollection(features)

    def _generar_rangos_mensuales(self):
        months = []
        current = datetime.strptime(self.start_date, '%Y-%m-%d')
        end     = datetime.strptime(self.end_date, '%Y-%m-%d')
        while current <= end:
            y, m = current.year, current.month
            first_day = datetime(y, m, 1)
            last_day  = datetime(y, m, calendar.monthrange(y, m)[1])
            if last_day > end:
                last_day = end
            months.append({
                'nombre': self.month_names[m-1],
                'inicio': first_day.strftime('%Y-%m-%d'),
                'fin':    last_day.strftime('%Y-%m-%d'),
                'anio':   y
            })
            # avanzar al mes siguiente
            current = datetime(y + int(m == 12), (m % 12) + 1, 1)
        return months

    def _escalar_bandas(self, image):
        return image.divide(10000)

    def _cargar_mediana_mensual_s2(self, inicio, fin):
        return (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                .filterBounds(self.roi_fc)
                .filterDate(inicio, fin)
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', self.max_cloud))
                .select(['B4', 'B3', 'B2'])
                .map(self._escalar_bandas)
                .median())

    def exportar_a_drive(self, folder='TRUCHAS'):
        for m in self.months:
            s2 = self._cargar_mediana_mensual_s2(m['inicio'], m['fin'])
            features_info = self.roi_fc.getInfo()['features']
            for f in features_info:
                geom = ee.Geometry(f['geometry'])
                fid  = f['properties']['id']
                desc = f"S2_{m['nombre']}_{m['anio']}_ID_{fid}"

                task = ee.batch.Export.image.toDrive(
                    image       = s2.clip(geom),
                    description = desc,
                    folder      = folder,
                    region      = geom.bounds().getInfo()['coordinates'],
                    scale       = 10,
                    crs         = 'EPSG:4326',
                    fileFormat  = 'GeoTIFF',
                    maxPixels   = 1e13
                )
                task.start()
                print(f"▶️ Iniciada exportación: {desc}")

In [12]:
# 1. GEOMETRÍAS RECTANGULARES EN UNA LISTA
coords_list = [
  [[[-69.13662367422387, -16.244924124906422],
    [-69.11319189626977, -16.244924124906422],
    [-69.11319189626977, -16.228525071734513],
    [-69.13662367422387, -16.228525071734513]]],
  [[[-69.1628878648977, -16.2576963094371],
    [-69.13773947317407, -16.2576963094371],
    [-69.13773947317407, -16.239238175858656],
    [-69.1628878648977, -16.239238175858656]]],
  [[[-69.2071765001516, -16.26684233145817],
    [-69.18812208730981, -16.26684233145817],
    [-69.18812208730981, -16.25242273383973],
    [-69.2071765001516, -16.25242273383973]]],
  [[[-69.2251151140432, -16.269149368839535],
    [-69.20880728323266, -16.269149368839535],
    [-69.20880728323266, -16.254647540803322],
    [-69.2251151140432, -16.254647540803322]]],
  [[[-69.24459868032739, -16.2693141561866],
    [-69.2258875902395, -16.2693141561866],
    [-69.2258875902395, -16.2537411410066],
    [-69.24459868032739, -16.2537411410066]]],
  [[[-69.27120619375512, -16.280354593297076],
    [-69.25163679678246, -16.280354593297076],
    [-69.25163679678246, -16.26395849661404],
    [-69.27120619375512, -16.26395849661404]]],
  [[[-69.31111746389672, -16.266595147273858],
    [-69.27249365408227, -16.266595147273858],
    [-69.27249365408227, -16.233304836403246],
    [-69.31111746389672, -16.233304836403246]]],
  [[[-69.34116837968982, -16.258111290799928],
    [-69.3130159138695,  -16.258111290799928],
    [-69.3130159138695,  -16.23866432078986],
    [-69.34116837968982, -16.23866432078986]]],
  [[[-69.43559377984296, -16.19188409838657],
    [-69.39971655205976, -16.19188409838657],
    [-69.39971655205976, -16.171276507969868],
    [-69.43559377984296, -16.171276507969868]]],
  [[[-69.39894407586347, -16.254765147422443],
    [-69.34195249871503, -16.254765147422443],
    [-69.34195249871503, -16.18702090096502],
    [-69.39894407586347, -16.18702090096502]]]
];

In [13]:
exportador = Sentinel2Exporter(
    coords_list=coords_list,
    start_date='2021-06-01',
    end_date='2021-09-30',
    max_cloud=12
)

exportador.exportar_a_drive()

▶️ Iniciada exportación: S2_Junio_2021_ID_1
▶️ Iniciada exportación: S2_Junio_2021_ID_2
▶️ Iniciada exportación: S2_Junio_2021_ID_3
▶️ Iniciada exportación: S2_Junio_2021_ID_4
▶️ Iniciada exportación: S2_Junio_2021_ID_5
▶️ Iniciada exportación: S2_Junio_2021_ID_6
▶️ Iniciada exportación: S2_Junio_2021_ID_7
▶️ Iniciada exportación: S2_Junio_2021_ID_8
▶️ Iniciada exportación: S2_Junio_2021_ID_9
▶️ Iniciada exportación: S2_Junio_2021_ID_10
▶️ Iniciada exportación: S2_Julio_2021_ID_1
▶️ Iniciada exportación: S2_Julio_2021_ID_2
▶️ Iniciada exportación: S2_Julio_2021_ID_3
▶️ Iniciada exportación: S2_Julio_2021_ID_4
▶️ Iniciada exportación: S2_Julio_2021_ID_5
▶️ Iniciada exportación: S2_Julio_2021_ID_6
▶️ Iniciada exportación: S2_Julio_2021_ID_7
▶️ Iniciada exportación: S2_Julio_2021_ID_8
▶️ Iniciada exportación: S2_Julio_2021_ID_9
▶️ Iniciada exportación: S2_Julio_2021_ID_10
▶️ Iniciada exportación: S2_Agosto_2021_ID_1
▶️ Iniciada exportación: S2_Agosto_2021_ID_2
▶️ Iniciada exportación: S2_